In [1]:
import threading
import time
import random

In [2]:
class ProveedorChicles(threading.Thread):
    def __init__(self, condition):
        threading.Thread.__init__(self)
        self.condition = condition
    
    def run(self):
        global chicles
        while True:
            self.condition.acquire()
            chicles.append(1)
            print ("Producido chicle: ", 1)
            self.condition.notify()
            self.condition.release()
            time.sleep(random.random())

In [3]:
class ProveedorPolvo(threading.Thread):
    def __init__(self, condition):
        threading.Thread.__init__(self)
        self.condition = condition
    
    def run(self):
        global polvos
        while True:
            self.condition.acquire()
            polvos.append(1)
            print ("Producido polvo: ", 1)
            self.condition.notify()
            self.condition.release()
            time.sleep(random.random())

In [4]:
class Cubridora(threading.Thread):
    def __init__(self, condition_chicles, condition_polvos, condition_bolichicles):
        threading.Thread.__init__(self)
        self.condition_chicles = condition_chicles
        self.condition_polvos = condition_polvos
        self.condition_bolichicles = condition_bolichicles
        self.inner_chicles = []
        self.inner_polvos = []
    
    def run(self):
        global polvos, chicles, bolichicles
        
        while True:
            self.condition_chicles.acquire()
            if not chicles:
                print ("Nada en la cola de chicles, cubridora esperando")
                self.condition_chicles.wait()
                print ("Chiclero agrego un chicle")
            self.inner_chicles.append(chicles.pop(0))
            
            print ("Consumido chicle", len(self.inner_chicles))
            self.condition_chicles.release()
        
#         while True:
            
            self.condition_polvos.acquire()
            if not polvos:
                print ("Nada en la cola de polvos, cubridora esperando")
                condition_polvos.wait()
                print ("Polvero agrego un chicle")
            self.inner_polvos.append(polvos.pop(0))
            print ("Consumido polvo", len(self.inner_polvos))
            self.condition_polvos.release()
        
#         while True:
            if self.inner_chicles and self.inner_polvos:
                self.condition_bolichicles.acquire()
                self.inner_chicles.pop(0)
                self.inner_polvos.pop(0)
                bolichicles.append(1)
                print ("Producido un bolichicle: ", 1)
                self.condition_bolichicles.notify()
                self.condition_bolichicles.release()
                time.sleep(random.random())
                
            
            time.sleep(random.random())

In [5]:
class Envolvedora(threading.Thread):
    def __init__(self, condition_bolichicles, condition_bolsas):
        threading.Thread.__init__(self)
        self.condition_bolichicles = condition_bolichicles
        self.condition_bolsas = condition_bolsas
        self.inner_bolichicles = 0
    
    def run(self):
        global bolichicles, bolsas
        while True:
            self.condition_bolichicles.acquire()
            if not bolichicles:
                print ("Nada en la cola, envolvedor esperando")
                condition_bolichicles.wait()
                print ("Productor agrego algo a la cola y notifico al consumidor")
            num = bolichicles.pop(0)
            print ("Consumido un bolichicle", num)
            self.inner_bolichicles += 1
            self.condition_bolichicles.release()
            time.sleep(random.random())
            
            if self.inner_bolichicles >= 10:
                self.inner_bolichicles -= 10 
                self.condition_bolsas.acquire()
                bolichicles.append(1)
                print ("Producida una bolsa", 1)
                self.condition_bolsas.notify()
                self.condition_bolsas.release()
                time.sleep(random.random())

In [6]:
class Envasadora(threading.Thread):
    def __init__(self, condition):
        threading.Thread.__init__(self)
        self.condition = condition
        self.contador_bolsas = 0
    
    def run(self):
        global bolsas
        while True:
            self.condition.acquire()
            if not bolsas:
                print ("Nada en la cola, envasador esperando")
                self.condition.wait()
                print ("Cubridora agrego algo a la cola y notifico al envasador")
            self.contador_bolsas += 1
            print ("Consumido una bolsa", self.contador_bolsas)
            self.condition.release()
            time.sleep(random.random())
    

In [7]:
if __name__ == '__main__':
    
    condition_chicles = threading.Condition()
    condition_polvos = threading.Condition()
    condition_bolichicles = threading.Condition()
    condition_bolsas = threading.Condition()

    chicles = []
    polvos = []
    bolichicles = []
    bolsas = []

    ProveedorChicles(condition_chicles).start()
    ProveedorPolvo(condition_polvos).start()
    Cubridora(condition_chicles, condition_polvos, condition_bolichicles).start()
    Envolvedora(condition_bolichicles, condition_bolsas).start()
    Envasadora(condition_bolsas).start()
    


Producido chicle:  1
Producido polvo:  1
Consumido chicle 1
Consumido polvo 1
Producido un bolichicle:  1
Consumido un bolichicle 1Nada en la cola, envasador esperando

Producido chicle:  1
Producido polvo:  1
Nada en la cola, envolvedor esperando
Producido polvo:  1
Consumido chicle 1
Consumido polvo 1
Producido un bolichicle:  1
Productor agrego algo a la cola y notifico al consumidor
Consumido un bolichicle 1
Producido chicle:  1
Producido polvo:  1
Nada en la cola, envolvedor esperando
Producido chicle:  1
Producido polvo:  1
Producido polvo:  1
Consumido chicle 1
Consumido polvo 1
Producido un bolichicle:  1
Productor agrego algo a la cola y notifico al consumidor
Consumido un bolichicle 1
Producido chicle:  1
Consumido chicle 1
Consumido polvo 1
Producido un bolichicle:  1
Producido polvo:  1
Consumido un bolichicle 1
Producido chicle:  1
Producido chicle:  1
Producido polvo:  1
Consumido chicle 1
Consumido polvo 1
Producido un bolichicle:  1
Producido chicle:  1
Consumido un bol

Consumido un bolichicle 1
Producido polvo:  1
Producido chicle:  1
Producido polvo:  1
Producido chicle:  1
Nada en la cola, envolvedor esperando
Producido chicle:  1
Producido polvo:  1
Producido polvo:  1
Consumido chicle 1
Consumido polvo 1
Producido un bolichicle:  1
Productor agrego algo a la cola y notifico al consumidor
Consumido un bolichicle 1
Producido chicle:  1
Consumido chicle 1
Consumido polvo 1
Producido un bolichicle:  1
Producido chicle:  1
Producido polvo:  1
Producido chicle:  1
Consumido un bolichicle 1
Producido chicle:  1
Producido chicle:  1
Producido chicle:  1
Producido polvo:  1
Producido chicle:  1
Producido polvo:  1
Consumido chicle 1
Consumido polvo 1
Producido un bolichicle:  1
Consumido un bolichicle 1
Consumido chicle 1
Consumido polvo 1
Producido un bolichicle:  1
Producido polvo:  1
Producido polvo:  1
Consumido chicle 1
Consumido polvo 1
Producido un bolichicle:  1
Producido chicle:  1
Consumido chicle 1
Consumido polvo 1
Producido un bolichicle:  1


Producido chicle:  1
Producido chicle:  1
Consumido un bolichicle 1
Producido polvo:  1
Producido chicle:  1
Producido polvo:  1
Nada en la cola, envolvedor esperando
Consumido chicle 1
Consumido polvo 1
Producido un bolichicle:  1
Productor agrego algo a la cola y notifico al consumidor
Consumido un bolichicle 1
Producido polvo:  1
Producido chicle:  1
Nada en la cola, envolvedor esperando
Producido chicle:  1
Producido polvo:  1
Producido chicle:  1
Producido polvo:  1
Consumido chicle 1
Consumido polvo 1
Producido un bolichicle:  1
Productor agrego algo a la cola y notifico al consumidor
Consumido un bolichicle 1
Producido chicle:  1
Nada en la cola, envolvedor esperando
Producido polvo:  1
Producido chicle:  1
Consumido chicle 1
Consumido polvo 1
Producido un bolichicle:  1
Productor agrego algo a la cola y notifico al consumidor
Consumido un bolichicle 1
Producido polvo:  1
Nada en la cola, envolvedor esperando
Producido chicle:  1
Consumido chicle 1
Consumido polvo 1
Producido un